In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os, gc
import pandas as pd
import numpy as np
import janestreet
import datatable as dt

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load


import tensorflow as tf
device = 'cuda' if tf.config.list_physical_devices('GPU') else 'cpu'
print(device)

tf.random.set_seed(197)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from catboost import Pool, CatBoostClassifier, cv, CatBoostRegressor

In [ ]:
train = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')
train = train.to_pandas()

In [ ]:
ignore_columns = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp','ts_id','target']
features = [col for col in train.columns if col not in ignore_columns]
print(features)

In [ ]:
%%time
f_mean = train[features[3:]].mean()
train[features[3:]] = train[features[3:]].fillna(f_mean)
train['action'] = (train['resp'] > 0).astype('int')
f_mean = np.array(f_mean)
np.save('f_mean.npy', f_mean)

In [ ]:
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                    
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [ ]:
%%time
train = reduce_memory_usage(train)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
df_train = train.sample(frac=0.9, random_state=0)
df_valid = train.drop(df_train.index)

In [ ]:
X_train = df_train.loc[:, df_train.columns.str.contains('feature')]
X_valid = df_valid.loc[:, df_valid.columns.str.contains('feature')]
y_train = df_train['action']
y_valid = df_valid['action']

In [ ]:
model = CatBoostClassifier(iterations=300, task_type="GPU", learning_rate=0.05, l2_leaf_reg=3.5, depth=11, loss_function= 'Logloss', eval_metric='AUC',use_best_model=True,random_seed=42)

In [ ]:
model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

In [ ]:
TRAINING = True

if TRAINING:
    import janestreet
    env = janestreet.make_env()
    th = 0.502
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt[:, 1:].sum()):
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:])
            pred = model.predict(x_tt)
            pred_df.action = np.where(pred >= th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)